In [70]:
import pathlib
import textwrap
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import json
import re
import threading
import queue
import time

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [71]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=var = os.environ['GOOGLE_API_KEY']
genai.configure(api_key=GOOGLE_API_KEY)

In [72]:
def parseTopic(rawtopics):
    topics=[]
    try:
        topics=json.loads(re.findall(r"\{.*\}",rawtopics.text)[0])["main_topics"]
    except Exception as e:
        pass
    finally:
        return topics

In [73]:
def getTopics(model,descr,repo=None,queue=None):
    ques=f"Can you extract a list of main topics from the following text and output it in  a json format ? for the json you should strictly follow the following format {{\"main_topics\":[]}}. Please ensure the output is in UTF-8 and json compliant. Please try to extract topic synthetic as possible. Here is the text from which extract topics: '{descr}'"
    resp = model.generate_content(ques)
    topics=parseTopic(resp)
    if(queue is not None):
        queue.put([repo]+topics)
    return topics

In [74]:
model = genai.GenerativeModel('gemini-1.5-flash')

In [75]:
desc=pd.read_csv("en_desc.csv",names=["oldidx","repo","desc"],header=0)
desc["repo"]=desc["repo"].apply(lambda x:x.strip())
desc["desc"]=desc["desc"].apply(lambda x:x.strip())

print(desc.shape)

(348891, 3)


In [76]:
q = queue.Queue()
workers=[]
#for idx in range(desc.shape[0]):
st=time.time()
for idx in range(100):
    reponame=desc.iloc[idx]["repo"]
    workers+=[threading.Thread(target=getTopics, args=(model,desc.iloc[idx]["desc"],reponame,q))]
    workers[-1].start()
    #workers[-1].join()
for w in workers:
    w.join()
print(f"execution time {time.time()-st}")

execution time 1.8834402561187744


In [68]:
print(q.qsize())
while not q.empty():
    print(q.get())

100
['sindresorhus/awesome', 'Interesting Topics', 'Lists']
['tensorflow/tensorflow', 'Open Source Machine Learning', 'Framework', 'Accessibility']
['trekhleb/javascript-algorithms', 'Algorithms', 'Data Structures', 'JavaScript']
['vinta/awesome-python', 'Python frameworks', 'Python libraries', 'Python software', 'Python resources']
['axios/axios', 'HTTP client', 'Promise-based', 'Browser', 'Node.js']
['donnemartin/system-design-primer', 'System Design', 'Interview Preparation', 'Anki Flashcards']
['kubernetes/kubernetes', 'Container Scheduling', 'Container Management', 'Production Environments']
['30-seconds/30-seconds-of-code', 'Code Snippets', 'Development']
['public-apis/public-apis', 'Free APIs', 'API Resources']
['jlevy/the-art-of-command-line', 'Command Line Mastery']
['codecrafters-io/build-your-own-x', 'Programming', 'Learning by Building', 'Project-Based Learning', 'Technology Recreation']
['puppeteer/puppeteer', 'Node.js', 'Chrome API']
['golang/go', 'Go programming language

In [84]:
alltopics=[]
for idx in tqdm(range(desc.shape[0])):
    reponame=desc.iloc[idx]["repo"]
    rawtopics=getTopics(model, desc.iloc[idx]["desc"])
    try:
        topics=json.loads(re.findall(r"\{.*\}",rawtopics.text)[0])["main_topics"]
    except Exception as e:
        pass
    alltopics+=[[reponame,",".join(topics)]]   
    
    geminiTopic = pd.DataFrame(np.array(alltopics),columns=["repo","topics"])
    geminiTopic.to_csv("geminiTopic.csv")

  1%|▉                                                                                                                       | 2835/348891 [47:59<97:38:38,  1.02s/it]


DeadlineExceeded: 504 Deadline Exceeded

In [99]:
topicold=pd.read_csv("out.csv")
topicold2=pd.read_csv("description.csv")

ParserError: Error tokenizing data. C error: Expected 3 fields in line 20, saw 5


In [97]:
len(list(set(topicold["topics"].tolist())))

69310

In [98]:
topicold.shape

(69532, 3)